# About

*by Dr Paul Richmond ([ICCS](https://iccs.cam.ac.uk/) Engineering Lead at University of Cambridge)*

This is an "Introduction to CUDA" lab designed to be executed inside a Jupyter notebook. It follows on from a series of lectures. You can use the notebook functionality to edit files and run code cells.

Some hints are provided in hidden markdown cells. If you are struggling with a particular exercise then click the three dots "..." to show the hint (if one is available).

*Note: If you are running this lab on Google Colab then you will need to run the following code cell to obtain the source files.*

In [ ]:
!git init .
!git remote add -f origin https://github.com/Cambridge-ICCS/CUDALabMorning.git
!git checkout main


## Exercise 01

Exercise 1 requires that we de-cipher some encrypted text. The text provided in the file [`encrypted01.bin`](./encryted01.bin) (click file to open in JupyterLab) has been encrypted by using an affine cipher. The affine cypher is a very simple type of monoalphabetic substitution cypher where each numerical character of the alphabet is encrypted using a mathematical function. The encryption function is defined as;

$E(x)=(Ax+B) mod M$

Where $A$ and $B$ are keys of the cypher, $mod$ is the modulo operation and $A$ and $M$ are co-prime. For this exercise the value of $A$ is `15`, $B$ is `27` and $M$ is `128` (the size of the ASCII alphabet). The affine decryption function is defined as:

$D(x)= A^{-1} (x-B)  mod M$

Where $A^{-1}$ is the modular multiplicative inverse of $A$ modulo $M$. For this exercise $A^{-1}$ has a value of `111`. 

Note: The $mod$ operation is not the same as the remainder operator (`%`) for negative numbers. A suitable $mod$ function (`modulo`) has been provided for the example. The provided function takes the form of `modulo(int a, int b)` where `a` in this case is everything left of the affine decryption function's $mod$ operator and `b` is everything to the right of the $mod$ operator.

As each of the encrypted character values are independent we can use the GPU to decrypt them in parallel. To do this we will launch a thread for each of the encrypted character values and use a kernel function to perform the decryption. Starting from the code provided in [`exercise01.cu`](./exercise01.cu) (click file to open and edit in JupyterLab), complete the following tasks;


### Step 1

Modify the `modulo` function by adding the correct function decorator so that it can be called on the device by the `affine_decrypt` kernel. 

*Hint:* Using the `__device__` decorator will ensure that the function will be compiled as device code and will then be available to call by CUDA Kernels and other device functions.

Although your code won't perform any decryption at this point you can try to build it by running the following cell. Alternatively you can open a new Terminal from the JupyterLab File menu and run the command yourself. It should produce some compiler warnings about "declared but never referenced" variables.


In [ ]:
!nvcc exercise01.cu -o exercise01

The above will compile and link the `exercise01.cu` file with the NVIDIA CUDA compiler (`nvcc`) and output (`-o`) the executable `exercise01`. At this stage the output buffer will only have junk within it but you can run the executable (in the code cell below) to confirm that it produces an output.

In [ ]:
!./exercise01

### Step 02

Implement the decryption kernel (`affine_decrypt`) for a single block of threads with an `x` dimension of `N` (`1024`). A kernel definition stub is already provided in the source file. The function has two arguments. The input is provided in `d_input`. You should perform your calculation and store the result in `d_output`. You can use the inverse modulus `A`, `B` and `M` C pre-processor definitions (at the top of the source file). 


*Hint:* You can read a single character from the input array by using the `threadIdx` as an index. E.g.

```
int value = d_input[threadIdx.x];
```

It is assumed that the block of threads is one dimensional and as such only the `x` value is required to obtain a unique index position. The same approach can be taken to write to a unique location in the output. The input value can be decrypted by calling the module function. E.g.

```
modulo(A_MMI_M * (value - B), M)
```



### Step 03

Allocate some memory on the device for the input (`d_input`) and output (`d_output`). A variable `size` exists already to calculate the size in bytes on the input and output array (of length `N`) 


*Hint:* `cudaMalloc` Expects and argument which is a pointer to a pointer. The first argument should therefore be the address of a pointer. E.g. `((void **)&d_input, size);`. The cast in this example will explicitly cast the address to the generic void type. Without this the cast would take place implicitly. 

### Step 04

1.4. Copy the host input values in `h_input` to the device memory `d_input`.

*Hint:* The function `cudaMemcpy` can be used to copy memory. The function always expects the destination first (as a pointer), followed by source (as a pointer), the size and a a transfer direction. E.g. 

```
cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
```

### Step 05

Configure a single one dimensional block of `N` threads using the `blocksPerGrid` and `threadsPerBlock` variables which are commented out in the starting code. Launch the `affine_decrypt` kernel using the kernel launch angle bracket notation `<<<>>>`.



*Hint:* As we initially want to launch a single block, `blocksPerGrid` can have the value `(1, 1, 1)`. Remember that the value one defines a size in each dimension (x, y, z). Similarly `threadsPerBlock` can be defined as `(N, 1, 1)`.

The kernel should be launched using the grid and then block configuration respectively. E.g. 

```
affine_decrypt_multiblock <<<blocksPerGrid, threadsPerBlock >>>(d_input, d_output);
```

It is essential that `d_input` and `d_output` are pointers to memory that has been allocated (using `cudaMalloc`) as memory on the device.

### Step 06

Copy the device output values in `d_output` to the host memory `h_output`.


*Hint:* The function `cudaMemcpy` can be used to copy memory. The function always expects the destination first (as a pointer), followed by source (as a pointer), the size and a a transfer direction. E.g.

```
cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);
```

### Step 07 

Compile and execute your program. You can do this using the code cells below. Alternatively you can open a new Terminal from the JupyterLab File menu. If you have completed the exercise correctly, you should decrypt the text.

In [ ]:
# compile
!nvcc exercise01.cu -o exercise01
# execute
!./exercise01

### Step 08

Don’t go running off through the forest just yet! Modify your code to complete the `affine_decrypt_multiblock` kernel which should work when using multiple blocks of threads. Change your grid and block dimensions so that you launch `8` blocks of `128` threads.


*Hint:* Implementing the multi-block approach requires that the grid block configuration is changed. E.g.

```
dim3 blocksPerGrid(8, 1, 1);
dim3 threadsPerBlock(N / 8, 1, 1);
```

The unique index for a thread within the kernel can then be computed as

```
int index = blockDim.x*blockIdx.x + threadIdx.x;
```

### Exercise 01 Solutions

If you have found that you have got stuck with the code you can view the solutions by checking out the solution file using the code cell below. The command will checkout a single file from the solutions branch of the repository but it will override any changes you have made.

In [ ]:
!git checkout origin/solutions -- exercise01.cu

## Exercise 02

In exercise 2 we are going to extend the vector addition example from the lecture. The file [`exercise02.cu`](./exercise02.cu) has been provided as a starting point (click file to open and edit in JupyterLab). Perform the following steps.

### Step 01

The code has an obvious mistake. Rather than correct it implement a CPU version of the vector addition (called `vectorAddCPU`) storing the result in an array called `c_ref`. Implement a new function `validate` which compares the GPU result to the CPU result. It should print an error for each value which is incorrect and return a value indicating the total number of errors. You should also print the number of errors to the console. Now fix the error and confirm your error check code works.

### Step 02

Change the value of `N` to `2050`. Do not run your code yet as it will now perform unsafe writes beyond the memory bounds which you have allocated. This is because a whole thread block is required for the extra two threads (our grid is always made up of entire blocks). You should modify the kernel by adding a check in the kernel so that you do not write beyond the bounds of the allocated memory. This will require you the ensure that the threads unique position that it indexed into memory does not exceed `N`. Threads which fail this test should no nothing. 

### Compile and Execute

You can compile and execute exercise 2 using the code cells below.

In [ ]:
# compile
!nvcc exercise02.cu -o exercise02

In [ ]:
# execute
!./exercise02

### Exercise 02 Solutions

If you have found that you have got stuck with the code you can view the solutions by checking out the solution file using the code cell below. The command will checkout a single file from the solutions branch of the repository but it will override any changes you have made.

In [ ]:
!git checkout origin/solutions -- exercise02.cu

## Exercise 03

We are going to implement a matrix addition kernel. In matrix addition, two matrices of the same dimensions are added entry wise. If you modify your code from exercise 2 by copying the file to a new file called [`exercise03.cu`](./exercise03.cu). It will require the following changes;


### Step 01

Modify the value of `size` so that you allocate enough memory for a matrix size of `N x N`. This will ensure the existing calls to `cudaMemcpy` copy the correct amount of memory. Set `N` to `2048`.

### Step 02

Modify the `random_ints` function to generate a random matrix rather than a vector. Note: The approach used to allocate memory for the 2D matrix uses a single pointer and as such indexing into each row will require a stride of `N`. E.g. To index row `j` and column `i` you would use `[j*N + i]`.

### Step 03

Rename your CPU implementation to `matrixAddCPU` and update the validate function for reference and to help test any errors with your kernel.

### Step 04

Change your launch parameters to launch a 2D grid of thread blocks with `256` threads per block. Rename the exiting `vectorAdd` kernel to `matrixAdd` and update the code to perform the matrix addition. 

*Hint:* You might find it helps to reduce `N` to a single thread block implementation initially.

### Compile and Execute

You can compile and execute exercise 2 using the code cells below.

In [ ]:
# compile
!nvcc exercise03.cu -o exercise03

In [ ]:
# execute
!./exercise03

### Exercise 03 Solutions

If you have found that you have got stuck with the code you can view the solutions by checking out the solution file using the code cell below. The command will checkout a single file from the solutions branch of the repository but it will override any changes you have made.

In [ ]:
!git checkout origin/solutions -- exercise03.cu